In [65]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import math
import re
from collections import Counter
#import itertools
import numpy as np
import plotly.express as px
import pandas as pd
import os



In [2]:
# https://alanhylands.com/how-to-web-scrape-wikipedia-python-urllib-beautiful-soup-pandas/
import urllib.request
url = "https://en.wikipedia.org/wiki/List_of_companies_of_Sweden"
page = urllib.request.urlopen(url)
from bs4 import BeautifulSoup
soup = BeautifulSoup(page, "lxml")
all_tables=soup.find_all('table', class_='wikitable sortable')



In [46]:
rows = all_tables[1].findAll('tr') # 'tr' = table row
header = rows[0]

colnames = ['link']
for col in header.findAll('th'):
    colnames.append(col.text.strip())

data_list = []
for row in rows:
    data_row = []
    try:
        link = row.find('a').get('href')
    except AttributeError:
        link = ''
    data_row.append(link)
    for col in row.findAll('td'):
        data_row.append(col.text.strip())
    data_list.append(data_row)
    
df = pd.DataFrame(data_list, columns = colnames)


In [48]:
df

,link,Name,Industry,Sector,Headquarters,Founded,Notes
0,,None,None,None,None,None,None
1,/wiki/3H_Biomedical,3H Biomedical,Health care,Biotechnology,Uppsala,2004,Cell-based biotech
2,/wiki/AarhusKarlshamn,AarhusKarlshamn,Consumer goods,Food products,Malmö,2005,"Vegetable oils, fats"
3,/wiki/Abba_Seafood,Abba Seafood,Consumer goods,Food products,Gothenburg,1883,Seafood
4,/wiki/ABU_Garcia,ABU Garcia,Consumer goods,Recreational products,Svängsta[5],1921,"Fishing reels, part of Newell Brands (US)"
...,...,...,...,...,...,...,...
298,/wiki/Wayne%27s_Coffee,Wayne's Coffee,Consumer services,Restaurants & bars,Stockholm,1994,Coffeehouse chain
299,/wiki/WESC,WESC,Consumer goods,Clothing & accessories,Stockholm,1999,Clothing
300,/wiki/WG_Film,WG Film,Consumer services,Broadcasting & entertainment,Malmö,1994,Film production
301,/wiki/WM-data,WM-data,Technology,Software,Stockholm,1969,"IT consulting, defunct 2008"


## Scrape

In this section, I scrape all wikipedia pages under the folowing link: 

In [66]:
def scrape_page(link):
    # https://stackoverflow.com/questions/52757547/how-to-select-whole-page-and-copy-using-selenium-python
    
    url = "https://en.wikipedia.org/" + link
    
    if os.path.isfile(link[1:] + ".txt"):
        return
    
    site = driver.get(url)
    element = driver.find_element(By.ID , 'bodyContent')
    try :
        with open(link[1:] + ".txt", "w") as text_file:
            text_file.write(element.text)
    except FileNotFoundError:
        pass
    return element


In [67]:
driver = webdriver.Chrome()
for i in range(1, df.shape[0]):
    time.sleep(0.3)
    link = df.iloc[i, 0]
    scrape_page(link)

## Calculate similarity score
Use cosine similarity to define how similar two bodies of text are.

In [3]:
WORD = re.compile(r"\w+")

def get_cosine(vec1, vec2):
    # https://stackoverflow.com/questions/15173225/calculate-cosine-similarity-given-2-sentence-strings
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x] ** 2 for x in list(vec1.keys())])
    sum2 = sum([vec2[x] ** 2 for x in list(vec2.keys())])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator

def text_to_vector(text):
    words = WORD.findall(text)
    return Counter(words)

In [4]:
# Loop over all pairs of pages to create the full matrix of cosine similarities
sim_matrix = np.zeros((num_pages-1, num_pages-1))
for i in range(1, num_pages):
    for j in range(1, num_pages):
        if i == j:
            continue
        with open('wikis/page{}.txt'.format(i)) as f:
            text1 = f.read()
        with open('wikis/page{}.txt'.format(j)) as f:
            text2 = f.read()

        vector1 = text_to_vector(text1)
        vector2 = text_to_vector(text2)

        cosine = get_cosine(vector1, vector2)
        sim_matrix[i-1, j-1] = cosine

In [5]:
fig = px.imshow(sim_matrix)
fig.show()

In [69]:
import plotly.express as px
